# 単語埋め込み

ディープラーニングが自然言語処理に活用され始める前は、単語を計算可能なベクトルに変換する処理としては、文書集合内の各単語がある文書に出現した場合に1を、そうでない場合は0を出すようなBag of Wordsというものがよく用いられた。これは扱うデータが大きく語彙数が多くなると非常に高次元になるため、特異値分解などで次元削減を行うこともあった。

ディープラーニングが活用されるようになり登場したものが単語の**埋め込み**（embedding）や**分散表現**（distributed representation）と呼ばれるもので、これらは$\{0, 1\}$に限らない値をとり、また次元数を語彙の数よりも小さく（次元圧縮）することもできる（実はニューラルネットワークを使った言語モデルの中間層を埋め込みとするので、それゆえ任意の次元数にできる）。

有名なのはWord2VecやElMoである。両者の大まかな違いとしては、Word2Vecは単語ごとに一意な分散表現になるが、ElMoは文脈によって分散表現が変わるということ。
例えば「彼はスポーツが下手だ」と「彼はいつも下手に出がちだ」は同じ「下手」という語だが意味が異なる。Word2Vecはこのような文脈を考慮しないが、ElMoは考慮する。

## 共起行列

文章$S = (w_1, \dots, w_d)$のある単語$w_i$の周囲の単語の集合（例えば両側$c$個をとって$C=\{ w_{i-c}, \dots, w_{i-1}, w_{i+1}, \dots, w_{i+c} \}$）を**文脈**（context）$C$といい、各単語$w_j (i \neq j)$が$C$に含まれるかどうかを$\{0, 1\}$で表現する。

この関係性を表す行列を**共起行列**（co-occurence matrix）という。

|       | $w_1$ | $w_2$ | $w_3$ | $w_4$ | $w_5$ |
|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|
| $w_3$ |   0   |   1   |   0   |   1   |   0   |



## 言語モデル

embeddingを直接推定するのではなく、単語予測モデル（言語モデル）を構築して副次的にembeddingを取得することになる。

言語モデルとは、尤もらしい文章を生成できるような確率分布を習得するために、文章の確率を推定するモデルのこと。

ある文章$S$をトークン化したのを$(w_1, w_2, \cdots, w_n)$と表記するならば、

$$
P(S)=P(w_1, w_2, \cdots, w_n)
$$

を求めたいということになる。これは条件付き確率の積として表せる

$$
\begin{align}
P(w_1, w_2, \cdots, w_n)
&= P(w_1) \times P(w_2|w_1) \times P(w_3|w_1, w_2) \times \cdots\\
&= \prod_{i=1}^n p(w_i|\boldsymbol{c}_i)
\end{align}
$$

ここで$\boldsymbol{c}_i$は$w_i$より前のトークン列$\boldsymbol{c}_i=(w_1,w_2,\cdots,w_n)$で、**文脈**（context）と呼ばれる



## Word2Vec

```
King - Man + Woman = Queen
```
のような語彙間の類似度や計算が可能な表現


### Word2Vecのアプローチは複数ある




Word2Vecの方法は、

1. CBOW (continuous bag of words)
2. Skip-Gram

の2種類あり、またそのままでは計算量が多すぎるために計算量削減の方法として

1. Hierarchical Softmax
2. Noise Constrastive Estimation
3. Negative Sampling

の方法があるため、組み合わせとしては2×3の6通りのモデルの組み方がある


### CBOW

**CBOW** (continuous bag of words)モデルは文章$S=(w_1, w_2, \dots, w_n)$が与えられた時、その$i$番目の単語$w_i$を、
その周りの単語である文脈$\boldsymbol{C}_i=(w_{i-c}, \dots, w_{i-1}, w_{i+1}, \dots, w_{i+c})$から予測するモデルである。ここで$c$はウィンドウサイズと呼ばれるハイパーパラメータ。







## gensimによる実装

https://radimrehurek.com/gensim/models/word2vec.html

In [146]:
# トークンに分割した文章の集合の例
sentences = [
    ["king", "male", "ruler"],
    ["queen", "female", "ruler"],
    ["man", "male"],
    ["woman", "female"],
]

from gensim.models import Word2Vec
model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, workers=4)

In [147]:
# kingに意味が近い上位3個（similarの基準はコサイン類似度）
sims = model.wv.most_similar('king', topn=3)
sims

[('male', 0.1459505707025528),
 ('woman', 0.041577354073524475),
 ('man', 0.03476494178175926)]

In [150]:
# king + woman - man ≒ queen というアレ
model.wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)

[('queen', 0.0066775488667190075)]

In [152]:
# 自前で足し引きして類似度計算してみる
import numpy as np
def cosine_sim(a, b):
    return a @ b / (np.linalg.norm(a) * np.linalg.norm(b))

v = model.wv["king"] - model.wv["man"] + model.wv["woman"]
words = set(sum(sentences, [])) - {"king", "man", "woman"} # 計算に使ったものは必然的に類似度が高くなっちゃうので除く
for word in words:
    print(f"{word}: {cosine_sim(v, model.wv[word]):.3g}")

# queenが一番近くなった

queen: 0.00656
ruler: -0.0263
female: -0.0395
male: -0.0251


### 参考文献

- https://code.google.com/archive/p/word2vec/
- [[1310.4546] Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)